# Week3_2 Assignment

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np

In [ ]:
!pip install tokenizers

In [ ]:
import torch
from tokenizers import BertWordPieceTokenizer

In [ ]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

cpu


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
cd # 데이터 다운로드할 위치 입력

In [ ]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
!pip install gdown
!gdown https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
!unzip processed.zip

In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [ ]:
with open(os.path.join(_DATA_DIR, "processed_wiki_ko.txt")) as reader:
    docs = reader.readlines()

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [ ]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # 악센트가 있는 character의 악센트를 제거 (ex. é → e)
    lowercase=False, # 한국어는 대소문자가 없기 때문에 lowercase = False
)

In [ ]:
# train
tokenizer.train(
    files = os.path.join(_DATA_DIR, "processed_wiki_ko.txt"),
    vocab_size = 30000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##",

)

In [ ]:
tokenizer.save_model(".", "wordpiece")

['./wordpiece-vocab.txt']

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [ ]:
tokenizer = BertWordPieceTokenizer(
    vocab = "./wordpiece-vocab.txt",
    lowercase=False,
    strip_accents=False,
)

In [ ]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

30000

In [ ]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
token_ids = tokenizer.encode(text).ids
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
print(tokenizer.encode(text).tokens)

[2, 7864, 20864, 16, 509, 3371, 5566, 2778, 5757, 16, 3]
['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']


In [ ]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text = "킹받네"
print(tokenizer.encode(text).tokens)
text = "열받네"
print(tokenizer.encode(text).tokens)

['[CLS]', '킹', '##받', '##네', '[SEP]']
['[CLS]', '열', '##받', '##네', '[SEP]']


In [ ]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "뷁뷁뷁"
unknown_token_ids = tokenizer.encode(text).ids
print(unknown_token_ids)
print(tokenizer.encode(text).tokens)

[2, 1, 3]
['[CLS]', '[UNK]', '[SEP]']


### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [ ]:
tokenizer.decode(token_ids)

'안녕하세요. 버트를 사용한 모델입니다.'

In [ ]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)

''